In [1]:
import cv2
import copy
import time
import tqdm
import pathlib
import random
import warnings
import datetime
import scipy as sp
import pandas as pd
import numpy as np
import xgboost as xgb
from time import time
import lightgbm as lgb
from catboost import Pool
from functools import partial
from catboost import CatBoost
from keras.models import Model
import matplotlib.pyplot as plt
from scipy.stats import laplace
from scipy.stats import kurtosis, skew
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy.optimize import curve_fit
from keras.layers import Dense, Input
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from collections import Counter, defaultdict
from scipy.interpolate import UnivariateSpline
from sklearn.preprocessing import MinMaxScaler
from hyperopt import hp, tpe, Trials, fmin, space_eval
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, average_precision_score, r2_score
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=5)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def mixture_laplace(x, loc1, scale1, coef1, loc2, scale2, coef2, loc3, scale3, coef3):
    coef1 = coef1 / (coef1 + coef2 + coef3)
    coef2 = coef2 / (coef1 + coef2 + coef3)
    coef3 = coef3 / (coef1 + coef2 + coef3)

    # scaleを非負にする
    scale1 = np.e ** scale1
    scale2 = np.e ** scale2
    scale3 = np.e ** scale3

    lap1 = laplace(loc1, scale1).pdf(x) * coef1
    lap2 = laplace(loc2, scale2).pdf(x) * coef2
    lap3 = laplace(loc3, scale3).pdf(x) * coef3
    # sin1 = sin_a * np.sin(sin_b * x + sin_c)
    return lap1 + lap2 + lap3

In [3]:
filepath = "../input/atma2020529/"
filepath2 = "../input/atma2020529-2/"
filepath3 = "../input/atma20205293/"
filepath4 = "../input/atam202005294/"
train = pd.read_csv(filepath + "train.csv")
test = pd.read_csv(filepath + "test.csv")
fitting = pd.read_csv(filepath4 + "fitting__fixed.csv")
sample_submission = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
spec_df = pd.read_csv(filepath3 + "spec.csv")
#wave_df = pd.read_csv(filepath2 + "wave_df.csv")
#wave_test = pd.read_csv(filepath2 + "wave_test.csv")
#wave_df = pd.DataFrame(wave_df)
#wave_test = pd.DataFrame(wave_test)
#wave_df = wave_df.iloc[:,:511]

# FE

In [4]:
from tsfresh import extract_features, extract_relevant_features
from tsfresh.feature_extraction import settings
# https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#module-tsfresh.feature_extraction.feature_calculators
fc_parameters = {
    "large_standard_deviation": [{ "r": 0.1}],   
    "ar_coefficient": [{"coeff": 3, "k": 10}], 
    "cid_ce" :[{"normalize": True}],
    "autocorrelation": [{"lag": 30}],
    "fft_aggregated" : [{"aggtype": "kurtosis"}],
    "fft_coefficient": [{"coeff": 3, "attr": "real"}],
}
ts_df = extract_features(spec_df, column_id="spectrum_filename", column_sort="wavelength", n_jobs=8, 
                         default_fc_parameters=fc_parameters) #settings.EfficientFCParameters())

ts_df = ts_df.reset_index()
ts_df = ts_df.rename(columns={"id":"spectrum_filename"})

Feature Extraction: 100%|██████████| 40/40 [00:14<00:00,  2.81it/s]


- "change_quantiles": [{"ql":0.8, "qh":1.0, "isabs": True, "f_agg": "var"}],
- "change_quantiles": [{"ql":0.6, "qh":1.0, "isabs": True, "f_agg": "var"}],
- "change_quantiles": [{"ql":0.4, "qh":1.0, "isabs": True, "f_agg": "var"}],
- "change_quantiles": [{"ql":0.0, "qh":1.0, "isabs": True, "f_agg": "var"}],
- "number_cwt_peaks": [{"n":10}],
- "c3": [{"lag": 40}],

In [5]:
def peak_near_stats(x):
    i = np.argmax(x)
    y = x[i - 10:i + 10]
    return np.sum(y) / x[i], kurtosis(y)

def peak_near_stats2(x):
    i = np.argsort(-x)[1]
    z = x[i - 5:i + 5]
    return kurtosis(z)

# aggragate spec information
spec_agg = spec_df.groupby("spectrum_filename")["intensity"].agg(["max", "min", "mean", "std"])
spec_agg.columns = ["intensity_" + c for c in spec_agg.columns]
fft = []
peak_near_ratio = []
peak_near1 = []
peak_near2 = []
for i, file_df in tqdm(spec_df.groupby("spectrum_filename")):
    x = file_df["wavelength"].values
    y = file_df["intensity"].values
        
    # fast fourier transformation
    F = np.fft.fft(y)
    Amp = np.abs(F)
    fft.append(np.quantile(Amp, 0.95))
    val1, val2 = peak_near_stats(y)
    peak_near_ratio.append(val1)
    peak_near1.append(val2)
    #second_largest.append(y[np.argsort(-y)[1]])
    #peak_near2.append(peak_near_stats2(y))
    
spec_agg["amp_0.95"] = fft
spec_agg["peak_near_ratio"] = peak_near_ratio
spec_agg["peak_near_1"] = peak_near1

In [6]:
def transform(df):
    new_df = df.copy()
    # merge original csv and fitting data
    new_df = pd.merge(new_df, fitting, on="spectrum_id", how="left")
    
    # merge original csv and spec information
    new_df= pd.merge(new_df, spec_agg.reset_index(), on="spectrum_filename", how="left")
    
    new_df = pd.merge(new_df, ts_df, on="spectrum_filename", how="left")
    
    # remove unnecessary columns
    if "target" in new_df.columns:
        new_df = new_df.drop(["spectrum_id",  "spectrum_filename"], axis=1)
    else:
        new_df = new_df.drop(["spectrum_id",  "spectrum_filename","chip_id"], axis=1)
    
    # create new variables
    new_df["ratio2_5"] = new_df["params2"] / new_df["params5"] # I don't know the meaning, but seems effective
    new_df["ratio3_1"] = new_df["params3"] / new_df["params1"] # I don't know the meaning, but seems effective
    #new_df["max1/2"] = new_df["intensity_max"] / new_df["second_largest"]
    
    return new_df
new_train = transform(train)
new_test = transform(test)
print(f'train shape: {new_train.shape}')
print(f'test shape: {new_test.shape}')

train shape: (7436, 31)
test shape: (6952, 29)


In [7]:
new_train.head()

,chip_id,exc_wl,layout_a,layout_x,layout_y,pos_x,target,params0,params1,params2,params3,params4,params5,params6,rms,beta,intensity_max,intensity_min,intensity_mean,intensity_std,amp_0.95,peak_near_ratio,peak_near_1,intensity__ar_coefficient__k_10__coeff_3,intensity__autocorrelation__lag_30,intensity__cid_ce__normalize_True,"intensity__fft_aggregated__aggtype_""kurtosis""","intensity__fft_coefficient__coeff_3__attr_""real""",intensity__large_standard_deviation__r_0.1,ratio2_5,ratio3_1
0,79ad4647da6de6425abf,850,2,36,140,1313.0810,0,30.808589,5.811802e+02,1037.714752,1.531423,22469.651641,1032.317268,8.295610,10.028668,2.521298e-02,1751.0,-228.0,40.292752,172.206792,7643.154333,0.000000,NaN,0.103792,0.025624,18.584115,8.347278,12341.143874,0.0,1.005229,2.635022e-03
1,79ad4647da6de6425abf,780,3,0,168,159.4150,0,91.300897,1.740582e+04,1080.510452,4.766233,33257.123175,1077.468855,8.018225,7.948485,3.435612e-01,4219.0,-263.0,166.958984,463.428363,28918.587810,9.569329,-1.257037,0.053777,0.025566,7.889624,5.127389,5877.317894,1.0,1.002823,2.738298e-04
2,c695a1e61e002b34e556,780,1,34,29,-610.7688,0,106.642946,1.000000e-10,1119.464438,2.000000,42579.867913,1378.883338,11.687417,10.739859,2.348528e-15,2412.0,-235.0,151.577691,327.857694,18130.782140,12.473051,-1.282283,0.233933,0.104350,10.395798,4.065969,32600.648173,1.0,0.811863,2.000000e+10
3,c695a1e61e002b34e556,780,2,32,139,1214.6180,0,306.933674,1.099486e+04,1139.855067,5.198692,39349.741703,1145.212849,9.445029,10.379948,2.183921e-01,3209.0,-52.0,523.080947,436.481410,22149.537147,12.990825,-1.479866,0.107048,0.092894,7.399576,4.660662,-22202.296078,1.0,0.995322,4.728291e-04
4,c695a1e61e002b34e556,780,0,45,85,-257.6161,0,46.133256,2.227622e+04,1120.918337,5.668012,31054.928673,1117.107782,7.658710,8.316550,4.176962e-01,3998.0,-245.0,138.187717,472.009931,30874.913315,10.321911,-1.386428,0.134784,0.015653,7.886848,4.748611,-45026.279667,1.0,1.003411,2.544423e-04


# modelling

In [8]:
categoricals = ['layout_a']
lgbm_params1 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.03, 'random_seed':44,'max_depth': 3}
lgbm_params2 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.03, 'random_seed':44,'max_depth': 3}
lgbm_params3 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.04, 'random_seed':44,'max_depth': 4}
lgbm_params4 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.045, 'random_seed':44,'max_depth': 4}
lgbm_params5 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.035, 'random_seed':44,'max_depth': 4}
lgbm_params6 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.04,  'random_seed':44,'max_depth': 3}

def modelling(new_train, new_test):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["chip_id"]))
    X_train["chip_id"] = lbl.transform(list(X_train["chip_id"]))
    
    remove_features = ["layout_x", "layout_y", "pos_x"]
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)
    
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
    X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

    n_folds=6
    skf=GroupKFold(n_splits = n_folds)
    models = []

    valid = np.array([])
    valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
    real = np.array([])
    features_list = [i for i in X_train.columns if i != "chip_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["chip_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        #print(X_train2.chip_id.unique())
        #print(lbl.inverse_transform(X_train2.chip_id.unique()))
        
        X_train2.drop(["chip_id"], axis=1, inplace=True)
        X_test2.drop(["chip_id"], axis=1, inplace=True)
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
        if i + 1 == 1:
            clf = lgb.train(lgbm_params1, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals) 
        elif i + 1 == 2:
            clf = lgb.train(lgbm_params2, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)  
        elif i + 1 == 3:
            clf = lgb.train(lgbm_params3, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)    
        elif i + 1 == 4:
            clf = lgb.train(lgbm_params4, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals) 
        elif i + 1 == 5:
            clf = lgb.train(lgbm_params5, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)
        else:
            clf = lgb.train(lgbm_params6, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)  
            
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid = np.concatenate([valid, valid_predict])
        valid_lgb.iloc[test_index]  = clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], 1)
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
        
        if i + 1 != 6:
            models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    score = average_precision_score(real, valid)
    print("average precision score = {}".format(average_precision_score(real, valid)))
    print(confusion_matrix(real, np.round(valid)))
    pred_value = np.zeros(X_test.shape[0])
    for i, model in enumerate(models):
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return score, pred_value, feature_importance_df, valid_lgb

# metric, pred_value, feature_importance_df, _ = modelling(new_train, new_test)

In [9]:
categoricals = ['layout_a']
lgbm_params1 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.035, 'random_seed':44,'max_depth': 3}
lgbm_params2 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.03, 'random_seed':44,'max_depth': 4}
lgbm_params3 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.04, 'random_seed':44,'max_depth': 4}
lgbm_params4 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.045, 'random_seed':44,'max_depth': 4}
lgbm_params5 = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.035, 'random_seed':44,'max_depth': 4}

def modelling_skf(new_train, new_test):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["chip_id"]))
    X_train["chip_id"] = lbl.transform(list(X_train["chip_id"]))
    
    remove_features = ["layout_x", "layout_y", "pos_x"]
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)
    
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
    X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds)
    models = []

    valid = np.array([])
    valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
    real = np.array([])
    features_list = [i for i in X_train.columns if i != "chip_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        
        X_train2.drop(["chip_id"], axis=1, inplace=True)
        X_test2.drop(["chip_id"], axis=1, inplace=True)
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
        if i + 1 == 1:
            clf = lgb.train(lgbm_params1, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals) 
        elif i + 1 == 2:
            clf = lgb.train(lgbm_params2, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)  
        elif i + 1 == 3:
            clf = lgb.train(lgbm_params3, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)    
        elif i + 1 == 4:
            clf = lgb.train(lgbm_params4, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals) 
        else:
            clf = lgb.train(lgbm_params5, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=40,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)
            
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid = np.concatenate([valid, valid_predict])
        valid_lgb.iloc[test_index]  = clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], 1)
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    score = average_precision_score(real, valid)
    print("average precision score = {}".format(average_precision_score(real, valid)))
    print(confusion_matrix(real, np.round(valid)))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return score, pred_value, feature_importance_df, valid_lgb

metric_skf, pred_value_skf, feature_importance_df_skf, _ = modelling_skf(new_train, new_test)

Fold 1
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[514]	training's pr_auc: 1	valid_1's pr_auc: 0.888038
Fold 2
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[410]	training's pr_auc: 1	valid_1's pr_auc: 0.803217
Fold 3
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[436]	training's pr_auc: 1	valid_1's pr_auc: 0.867641
Fold 4
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[199]	training's pr_auc: 0.999945	valid_1's pr_auc: 0.931646
Fold 5
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[291]	training's pr_auc: 1	valid_1's pr_auc: 0.890541
average precision score = 0.869309882243297
[[7176   24]
 [  71  165]]


In [10]:
feature_importance_df_skf.sort_values("Average", ascending=False).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,peak_near_1,273,481,542,237,333,373.2,118.592411,0.317772
1,ratio3_1,268,336,374,204,231,282.6,63.654065,0.225244
2,params3,200,282,438,175,184,255.8,98.671982,0.385739
3,intensity__ar_coefficient__k_10__coeff_3,200,296,380,147,224,249.4,80.992839,0.324751
4,peak_near_ratio,216,332,250,160,207,233.0,57.243340,0.245680
5,params6,238,223,299,178,156,218.8,49.853385,0.227849
6,rms,230,317,220,90,189,209.2,73.193989,0.349876
7,ratio2_5,178,200,245,119,243,197.0,46.634751,0.236725
8,intensity_max,192,224,223,123,218,196.0,38.319708,0.195509
9,params2,164,237,231,158,186,195.2,33.078089,0.169457


# submission

In [11]:
final_pred = pred_value_skf
score = metric_skf
sample_submission["target"] = final_pred
sample_submission.to_csv("atmacup3_sample_submission"+str(score)[:-10]+".csv", index = False)
sample_submission.head()

,target
0,0.000114
1,0.000127
2,0.000133
3,0.000031
4,0.012152
